#### This is the bonus part. In order to have an interactive environment to discuss the model performance, I got the permission from TA and submitted an ipynb file instead of python file.

In [1]:
!pip install pandas
!pip install keras
!pip install seaborn

import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model

from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, auc, mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold

from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

%matplotlib inline

## STEP1  Import data and do featurization

In [2]:
Salary = pd.read_csv('./Kaggle_Salary.csv')

/home/hanwen/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,3,9,11,13,22,24,25,26,27,28,29,45,57,65,84,86,88,108,110,124,126,151,195,209,224,250,263,265,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,305,307,323,326,327,330,342,372,385,386,394,395) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Drop three irrelavent columns
Salary.drop(['Unnamed: 0','index'], axis=1,inplace=True)

QuestionMapping = {}
for col in Salary.columns:
    QuestionMapping[col] = Salary.loc[0,col]
Salary.drop([0],inplace=True)
# print(Salary.shape)
Salary.head()

# See how many null values are in each column
# Here we only deal with those colunmns that has null values more than zero but fewer that 1% of all data
less_001 = []
for col in Salary.columns:
    NullNum = Salary[col].isnull().sum()
    if (NullNum > 0) & (0.01 * Salary.shape[0] > NullNum):
        if ("Part" not in col) or (col in ['Q39_Part_1','Q39_Part_2']):
            less_001.append(col)
# print("We drop nan rows in the below columns:",less_001)

for col in less_001:
    Salary.dropna(subset=[col],inplace=True)
# print(Salary.shape)
between_001_020 = []
for col in Salary.columns:
    NullNum = Salary[col].isnull().sum()
    if (0.2 * Salary.shape[0] > NullNum) and (NullNum >= 0.01 * Salary.shape[0]):
        if ("Part" not in col) or (col in ['Q39_Part_1','Q39_Part_2']):
            between_001_020.append(col)
# print("We use modes in these columns to replace nan value:",between_001_020)
for col in between_001_020:
    Salary[col].fillna(Salary[col].mode()[0],inplace=True)

between_020_050 = []
for col in Salary.columns:
    NullNum = Salary[col].isnull().sum()
    if (0.2 * Salary.shape[0]) < NullNum and (NullNum < 0.5 * Salary.shape[0]):
        if ("Part" not in col) or (col in ['Q39_Part_1','Q39_Part_2']):
            between_020_050.append(col)
# print("We set nans in these columns as 'Unknown':",between_020_050)

for col in between_020_050:
    Salary.loc[Salary[col].isnull(),col] = 'Unknown'
more_050 = []
for col in Salary.columns:
    NullNum = Salary[col].isnull().sum()
    if 0.5 * Salary.shape[0] < NullNum:
        if ("Part" not in col) or (col in ['Q39_Part_1','Q39_Part_2']):
            more_050.append(col)
# print("We drop these columns because large part of the data is missing:",more_050)

for col in more_050:
    Salary.drop([col], axis=1,inplace=True)
ALLCOL = Salary.columns.tolist()
def select_col(condition, allcol=ALLCOL):
    digit = len(condition)
    selected = []
    for i in allcol:
        if condition in i:
            if len(i)==digit:
                selected.append(i)
            elif i[digit] not in '0123456789':
                selected.append(i)                
    return selected
DummyCols = ['Q1','Q4','Q5','Q6','Q7','Q10','Q12_MULTIPLE_CHOICE',\
            'Q17','Q18','Q20','Q23','Q26','Q32','Q39_Part_1', 'Q39_Part_2',\
             'Q40','Q48']
for col in DummyCols:
    Salary = pd.get_dummies(data=Salary, columns=[col])
# print(Salary['Q3'].value_counts())
val_counts = Salary['Q3'].value_counts()
value_mask = Salary['Q3'].isin(val_counts.index[val_counts < 50])
Salary.loc[value_mask,'Q3'] = "Other"

Salary = pd.get_dummies(data=Salary, columns=['Q3'])
# print(Salary['Q22'].value_counts())
val_counts = Salary['Q22'].value_counts()
value_mask = Salary['Q22'].isin(val_counts.index[val_counts < 100])
Salary.loc[value_mask,'Q22'] = "Other"

Salary = pd.get_dummies(data=Salary, columns=['Q22'])
def Range_Normalize(value):
    if isinstance(value, str):
        if '-' in value:
            temp = value.split('-')
            return (float(temp[0])+float(temp[1]))/2
        if '+' in value:
            a = value.split('+')
            return float(a[0])
        else:
            return value
    else:
        return value
# Salary[select_col('Q2')]
Salary['Q2_NORMAL'] = Salary['Q2'].apply(Range_Normalize)
Salary.drop(['Q2'],axis=1,inplace=True)
# Salary[select_col('Q8')]
Salary['Q8_NORMAL'] = Salary['Q8'].apply(Range_Normalize)
Salary.drop(['Q8'],axis=1,inplace=True)
# Salary[select_col('Q24')]
def Q24Range_Normalize(value):
    mapdict = {'I have never written code and I do not want to learn':-1,
               'I have never written code but I want to learn':0,'< 1 year':0.5,
              '1-2 years':1.5,'3-5 years':4,'5-10 years':7.5,'10-20 years':15,
               '20-30 years':25,'30-40 years':35,'40+ years':40}
    return mapdict[value]
# Salary[select_col('Q2')]
Salary['Q24_NORMAL'] = Salary['Q24'].apply(Q24Range_Normalize)
Salary.drop(['Q24'],axis=1,inplace=True)
# Salary[select_col('Q25')]
Salary['Q25'].unique()
def Q25Range_Normalize(value):
    mapdict = {'I have never studied machine learning and I do not plan to':-1,
               'I have never studied machine learning but plan to learn in the future':0,
               '< 1 year':0.5,'1-2 years':1.5,'2-3 years':2.5,'3-4 years':3.5,'4-5 years':4.5,
               '5-10 years':7.5,'10-15 years':12.5,'20+ years':20}
    return mapdict[value]

Salary['Q25_NORMAL'] = Salary['Q25'].apply(Q25Range_Normalize)
Salary.drop(['Q25'],axis=1,inplace=True)
def Q41Range_Normalize(value):
    mapdict = {'Very important':10,
               'Slightly important':5,
               'No opinion; I do not know':2,
               'Unknown':0,
               'Not at all important':-10}
    return mapdict[value]

Salary.loc[Salary['Q41_Part_1'].isnull(),'Q41_Part_1'] = 'Unknown'
Salary.loc[Salary['Q41_Part_2'].isnull(),'Q41_Part_2'] = 'Unknown'
Salary.loc[Salary['Q41_Part_3'].isnull(),'Q41_Part_3'] = 'Unknown'

Salary['Q41_Part_1_NORMAL'] = Salary['Q41_Part_1'].apply(Q41Range_Normalize)
Salary.drop(['Q41_Part_1'],axis=1,inplace=True)
Salary['Q41_Part_2_NORMAL'] = Salary['Q41_Part_2'].apply(Q41Range_Normalize)
Salary.drop(['Q41_Part_2'],axis=1,inplace=True)
Salary['Q41_Part_3_NORMAL'] = Salary['Q41_Part_3'].apply(Q41Range_Normalize)
Salary.drop(['Q41_Part_3'],axis=1,inplace=True)
def Q43Range_Normalize(value):
    if isinstance(value, str):
        if '-' in value:
            temp = value.split('-')
            return (float(temp[0])+float(temp[1]))/2
        elif value == "0":
            return 0
        else:
            return float("nan")
    else:
        return value
Salary['Q43_NORMAL'] = Salary['Q43'].apply(Q43Range_Normalize)
Salary.drop(['Q43'],axis=1,inplace=True)
Salary['Q43_NORMAL'].fillna(Salary['Q43_NORMAL'].mode()[0],inplace=True)

Salary['Q46_NORMAL'] = Salary['Q46'].apply(Q43Range_Normalize)
Salary.drop(['Q46'],axis=1,inplace=True)
Salary['Q46_NORMAL'].fillna(Salary['Q46_NORMAL'].mode()[0],inplace=True)
# Salary[select_col('Q11')]
import math
def Binarize(value):
    if isinstance(value,str):
        return 1
    else:
        if value == 0 or math.isnan(value):
            return 0
        else:
            return 1
def binarize_col(columns, data=Salary):
    for col in columns:
        data[col] = data[col].apply(Binarize)
NormalCols = select_col('Q11_Part') + select_col('Q13_Part') + \
select_col('Q14_Part') + select_col('Q15_Part') + select_col('Q16_Part')\
+ select_col('Q19_Part') + select_col('Q21_Part') + select_col('Q27_Part')\
+ select_col('Q28_Part') + select_col('Q29_Part') + select_col('Q30_Part') +\
select_col('Q31_Part') + select_col('Q33_Part') + select_col('Q34_Part') +\
select_col('Q36_Part') + select_col('Q38_Part') + select_col('Q42_Part') + \
select_col('Q44_Part') + select_col('Q45_Part') + select_col('Q47_Part') + \
select_col('Q49_Part') + select_col('Q50_Part')
binarize_col(NormalCols)
def TBinarize(value):
    if isinstance(value,str):
        return float(value)
    else:
        if math.isnan(value):
            return -1
        else:
            return value
def Tbinarize_col(columns, data=Salary):
    for col in columns:
        data[col] = data[col].apply(TBinarize)

Tbinarize_col(select_col('Q34_Part'))
Tbinarize_col(select_col('Q35_Part'))
for col in Salary.columns:
    if isinstance(Salary.loc[1,col],str):
        Salary[col] = Salary[col].apply(lambda x:float(x))
Salary.rename(columns={"Q9": "Yearly_compensation"},inplace=True)
Salary.rename(columns={"Time from Start to Finish (seconds)": "TimeUse"},inplace=True)
Salary.drop(Salary[Salary.TimeUse < 180].index,inplace=True)
for col in Salary.columns:
    if len(Salary[col].value_counts())==1:
        Salary.drop([col], axis=1,inplace=True)
Salary.drop(Salary[Salary.Yearly_compensation < 1000].index, inplace=True)
Salary.drop(Salary[Salary.Yearly_compensation > 350000].index, inplace=True)
X = Salary.drop(['Yearly_compensation'],axis=1)
Y = Salary['Yearly_compensation']

from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X,Y,test_size=0.3, random_state=20)
print(X.shape,X_Train.shape)

(13995, 605) (9796, 605)


Since Neural Network is complex model with much parameters. Here we use all the features after featurization which have a dimension 605. Before feeding to the model, we standardized data.

In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
def feature_scaling(X_Train, X_Test, method):
    methodmap = {'z-score':StandardScaler(),'min-max':MinMaxScaler(),'robust':RobustScaler()}
    if method not in ['z-score','min-max','robust']:
        print("We don't have this kind of method for normalization. Please choose from ['z-score','min-max','robust']")
        return pd.DataFrame(X_Train), pd.DataFrame(X_Test)
    else:
        ss = methodmap[method].fit(X_Train)
        X_Train = ss.transform(X_Train)
        X_Test = ss.transform(X_Test)
        return pd.DataFrame(X_Train), pd.DataFrame(X_Test)
X_Train_All, X_Test_All = feature_scaling(X_Train,X_Test,'z-score')
print(X_Train_All.shape)

(9796, 605)


## STEP2  Develop a Baseline Neural Network Model

Below we create the baseline model to be evaluated. It is a simple model that has two single fully connected hidden layers with the same number of neurons as input attributes (605). The network uses good practices such as the rectifier activation function for the hidden layer. No activation function is used for the output layer because it is a regression problem and we are interested in predicting numerical values directly without transform. The efficient ADAM optimization algorithm is used and a mean squared error loss function is optimized.

In [5]:
# Learner(Vanilla/Basic Model)
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

/home/hanwen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [6]:
model1 = Sequential()

# Stack of Layers
model1.add(Dense(units=600, kernel_initializer='normal', activation='relu', input_dim=605))
model1.add(Dense(units=500, kernel_initializer='normal', activation='relu'))
model1.add(Dense(units=1, kernel_initializer='normal'))

model1.compile(loss='mean_squared_error',optimizer='adam') 

Kmod = model1.fit(X_Train_All,Y_Train, epochs=2000, batch_size=20, verbose=0)
Error = model1.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model1.predict(X_Test_All)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))
MAE = mean_absolute_error(Y_Test,Y_Test_pred)

print(RMSE,MAE)


Loss(Based on Testing Set): 1141378709.1555133
33784.29693975253 22446.168089567524


In [16]:
Y_Train_pred = model1.predict(X_Train_All)
RMSE = np.sqrt(mean_squared_error(Y_Train,Y_Train_pred))
MAE = mean_absolute_error(Y_Train,Y_Train_pred)

print(RMSE,MAE)

823.3228728623982 544.2172392023101


## STEP3  Tune The Neural Network Topology
There are many concerns that can be optimized for a neural network model. The point of biggest leverage maybe the structure of the network itself, including the number of layers and the number of neurons in each layer.

### 3.1 Evaluate a Deeper Network Topology

In this section we evaluate two additional network topologies in an effort to further improve the performance of the model. We look at deeper network topology.

In [7]:
model2 = Sequential()

# Stack of Layers
model2.add(Dense(units=600, kernel_initializer='normal', activation='relu', input_dim=605))
model2.add(Dense(units=500, kernel_initializer='normal', activation='relu'))
model2.add(Dense(units=400, kernel_initializer='normal', activation='relu'))
model2.add(Dense(units=1, kernel_initializer='normal'))

model2.compile(loss='mean_squared_error',optimizer='adam') 

Kmod = model2.fit(X_Train_All,Y_Train, epochs=2000, batch_size=20, verbose=0)
Error = model2.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model2.predict(X_Test_All)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))
MAE = mean_absolute_error(Y_Test,Y_Test_pred)

print(RMSE,MAE)


Loss(Based on Testing Set): 1114005714.4729698
33376.72416123762 21768.40713815673


In [17]:
Y_Train_pred = model2.predict(X_Train_All)
RMSE = np.sqrt(mean_squared_error(Y_Train,Y_Train_pred))
MAE = mean_absolute_error(Y_Train,Y_Train_pred)

print(RMSE,MAE)

1140.4459384107058 738.7390327998793


In [6]:
model3 = Sequential()

# Stack of Layers
model3.add(Dense(units=600, kernel_initializer='normal', activation='relu', input_dim=605))
model3.add(Dense(units=500, kernel_initializer='normal', activation='relu'))
model3.add(Dense(units=400, kernel_initializer='normal', activation='relu'))
model3.add(Dense(units=300, kernel_initializer='normal', activation='relu'))
model3.add(Dense(units=1, kernel_initializer='normal'))

model3.compile(loss='mean_squared_error',optimizer='adam') 

Kmod = model3.fit(X_Train_All,Y_Train, epochs=2000, batch_size=20, verbose=0)
Error = model3.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model3.predict(X_Test_All)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))
MAE = mean_absolute_error(Y_Test,Y_Test_pred)

print(RMSE,MAE)


Loss(Based on Testing Set): 1145437108.6296737
33844.30691874449 21582.245471537353


In [11]:
Y_Train_pred = model3.predict(X_Train_All)
RMSE = np.sqrt(mean_squared_error(Y_Train,Y_Train_pred))
MAE = mean_absolute_error(Y_Train,Y_Train_pred)

print(RMSE,MAE)

770.277671549459 502.40806438777827


We develop three models in total with different number of layers (3, 4 and 5). After 2000 epoches of training, we apply the model to training and testing set respectively and compute the RMSE and MAE.

| Model-layers |Test-RMSE   |Test-MAE    |Train-RMSE  |Train-MAE   |
|---           |---         |---         |---         |---         | 
|Three-Layer   |33784.296939|22446.168089|823.322872  |544.217239  |
|Four-Layer    |33376.724161|21768.407138|1140.445938 |738.739032  |
|Five-Layer    |33844.306918|21582.245471|770.277671  |502.408064  |

From the above table, we can see that for models with shallow layers, the gap between training performance and testing performance is smaller. There is a large gap between training and testing performance for model with five layers. The model with deep layers suffers from overfitting problem.

Meanwhile there isn't much difference for testing performance with repect to layers changes.

### 3.2 Evaluate a Narrow Network Topology

In this section we evaluate two additional network topologies to check how the number of nodes per layer influnce the model's performance.

In [7]:
model4 = Sequential()

# Stack of Layers
model4.add(Dense(units=500, kernel_initializer='normal', activation='relu', input_dim=605))
model4.add(Dense(units=400, kernel_initializer='normal', activation='relu'))
model4.add(Dense(units=300, kernel_initializer='normal', activation='relu'))
model4.add(Dense(units=200, kernel_initializer='normal', activation='relu'))
model4.add(Dense(units=1, kernel_initializer='normal'))

model4.compile(loss='mean_squared_error',optimizer='adam') 

Kmod = model4.fit(X_Train_All,Y_Train, epochs=2000, batch_size=20, verbose=0)
Error = model4.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model4.predict(X_Test_All)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))
MAE = mean_absolute_error(Y_Test,Y_Test_pred)

print(RMSE,MAE)


Loss(Based on Testing Set): 1123073420.4982138
33512.28760108963 21589.113500752712


In [12]:
Y_Train_pred = model4.predict(X_Train_All)
RMSE = np.sqrt(mean_squared_error(Y_Train,Y_Train_pred))
MAE = mean_absolute_error(Y_Train,Y_Train_pred)

print(RMSE,MAE)

651.1331129392292 396.2108799477605


In [8]:
model5 = Sequential()

# Stack of Layers
model5.add(Dense(units=400, kernel_initializer='normal', activation='relu', input_dim=605))
model5.add(Dense(units=300, kernel_initializer='normal', activation='relu'))
model5.add(Dense(units=200, kernel_initializer='normal', activation='relu'))
model5.add(Dense(units=100, kernel_initializer='normal', activation='relu'))
model5.add(Dense(units=1, kernel_initializer='normal'))

model5.compile(loss='mean_squared_error',optimizer='adam') 

Kmod = model5.fit(X_Train_All,Y_Train, epochs=2000, batch_size=20, verbose=0)
Error = model5.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model5.predict(X_Test_All)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))
MAE = mean_absolute_error(Y_Test,Y_Test_pred)

print(RMSE,MAE)


Loss(Based on Testing Set): 1121383894.1462252
33487.070664477134 21763.82505629363


In [13]:
Y_Train_pred = model5.predict(X_Train_All)
RMSE = np.sqrt(mean_squared_error(Y_Train,Y_Train_pred))
MAE = mean_absolute_error(Y_Train,Y_Train_pred)

print(RMSE,MAE)

1050.0625616956927 745.5063912082858


We use the same number of layers and continuously shrink the size of layers in the model. The number of nodes in three models are: (600,500,400,300,1) (500,400,300,200,1) (400,300,200,100,1). After 2000 epoches of training, we apply the model to testing set and compute the RMSE and MAE.
    
| Model-nodes    |Test-RMSE   |Test-MAE    |Train-RMSE  |Train-MAE   |
|---             |---         |---         |---         |---         |
|Most-nodes      |33844.306918|21582.245471|770.277671  |502.408064  |
|Medium-nodes    |33512.287601|21589.113500|651.133112  |396.210879  |
|Fewest-nodes    |33487.070664|21763.825056|1050.062561 |745.506391  |

From the above table, we can see that there is large gap between training and testing performance in three models. With the number of nodes increases, the performance is getting worse.

### 3.3 Evaluate a different activation function
We change original activation function from "relu" to "tanh" to check its performance.

In [9]:
model6 = Sequential()

# Stack of Layers
model6.add(Dense(units=600, kernel_initializer='normal', activation='tanh', input_dim=605))
model6.add(Dense(units=500, kernel_initializer='normal', activation='tanh'))
model6.add(Dense(units=400, kernel_initializer='normal', activation='tanh'))
model6.add(Dense(units=300, kernel_initializer='normal', activation='tanh'))
model6.add(Dense(units=1, kernel_initializer='normal'))

model6.compile(loss='mean_squared_error',optimizer='adam') 

# Training of Model(Train Data is splitted into 80/20)
Kmod = model6.fit(X_Train_All,Y_Train, epochs=2000, batch_size=20, verbose=0)
Error = model6.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model6.predict(X_Test_All)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))
MAE = mean_absolute_error(Y_Test,Y_Test_pred)

print(RMSE,MAE)


Loss(Based on Testing Set): 1349294330.5434628
36732.741703160726 23782.268511691072


In [14]:
Y_Train_pred = model6.predict(X_Train_All)
RMSE = np.sqrt(mean_squared_error(Y_Train,Y_Train_pred))
MAE = mean_absolute_error(Y_Train,Y_Train_pred)

print(RMSE,MAE)

8977.795195235367 3502.110539259936


We modify the activation function at each layer from 'relu' to 'tanh' and keep other structure the same. After 2000 epoches of training, we apply the model to testing set and compute the RMSE and MAE.
    
| Model-acti|Test-RMSE   |Test-MAE    |Train-RMSE  |Train-MAE   |
|---        |---         |---         |---         |---         |
|tanh       |36732.741703|23782.268511|8977.795195 |3502.110539 |
|relu       |33844.306918|21582.245471|770.277671  |502.408064  |

From the above table, we can see that the gap between training and testing is smaller with 'tanh' activation function than with 'relu'. Despite this, the 'relu' model also works better. The biggest advantage of relu is the non-saturation of its gradient, which greatly accelerates the convergence of stochastic gradient descent compared to tanh functions. Another nice property is that compared to tanh neurons that involve expensive operations (exponentials, etc.), the relu can be implemented by simply thresholding a matrix of activations at zero.

## 3.4 Evaluate a different optimization function
We change original optimization function from "adam" to "sgd" to check its performance.

In [10]:
model7 = Sequential()

# Stack of Layers
model7.add(Dense(units=600, kernel_initializer='normal', activation='relu', input_dim=605))
model7.add(Dense(units=500, kernel_initializer='normal', activation='relu'))
model7.add(Dense(units=400, kernel_initializer='normal', activation='relu'))
model7.add(Dense(units=300, kernel_initializer='normal', activation='relu'))
model7.add(Dense(units=1, kernel_initializer='normal'))

model7.compile(loss='mean_squared_error',optimizer='sgd') 

Kmod = model7.fit(X_Train_All,Y_Train, epochs=2000, batch_size=20, verbose=0)
Error = model7.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model7.predict(X_Test_All)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))
MAE = mean_absolute_error(Y_Test,Y_Test_pred)

print(RMSE,MAE)


Loss(Based on Testing Set): 2641556707.9552274
51396.07667195328 38918.68632784591


In [15]:
Y_Train_pred = model7.predict(X_Train_All)
RMSE = np.sqrt(mean_squared_error(Y_Train,Y_Train_pred))
MAE = mean_absolute_error(Y_Train,Y_Train_pred)

print(RMSE,MAE)

51778.060562684725 39101.64227108003


We modify the optimization function and keep other structures the same. After 2000 epoches of training, we apply the model to testing set and compute the RMSE and MAE.
    
| Model-opti|Test-RMSE   |Test-MAE    |Train-RMSE  |Train-MAE   |
|---        |---         |---         |---         |---         |
|sgd        |51396.076671|38918.686327|51778.060562|39101.642271|
|adam       |33844.306918|21582.245471|770.277671  |502.408064  |

From the above table, we can see that model with 'adam' optimization function works better than 'sgd' function. Adam updates its gradient by taking into account the previous gradient, which is called utilizing the momentum. Momentum can give faster convergence. Momentum simply means that some fraction of the previous update is added to the current update, so that repeated updates in a particular direction compound. Therefore, momentum would help to damp the oscillations in right directions.

For this problem, we can see from the results that a simple neural network can beat the linear models because the variables in the dataset isn't strictly linearly related with YearlyCompensation. However, Neural Network has no real Theory that explains how to choose the number of Hidden Layers except the Thumb Rule and Trial & Error procedure. It takes a lot of time when the Input Data is large, needs powerful computing machines. It is difficult to interpret the results and very hard to interpret and measure the impact of individual predictors. Neural Network also has an Inherent Tendency to overfit.